In [1]:
# Alp Ozkayikci Nezir Uran Jennifer Warbeck


In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

In [2]:
start_server_and_interface(scene_name="dragonfly_tadpole_scene")


The 'start_server_and_interface' function is not supported on Windows OS
Instead, start the server and interface by running the following command from the root directory in a Windows Powershell (make sure to activate the virtual environment before). Then click on the link to open the web interface:

start_all.bat dragonfly_tadpole_scene


In [3]:
controller = NotebookController()

In [4]:
controller.agents

In [5]:
controller.run()

In [6]:
def tadpole_behavior(agent):
    """Tadpole slows down when sensing a dragonfly larvae."""
    
    left, right = agent.sensors(sensed_entities=["DRAGONFLY_LARVAE"])
    
    left_motor = 0.5
    right_motor = 0.5
    if left > 0 or right > 0:
        left_motor = 0
        right_motor = 0
    return left_motor, right_motor

def dragonfly_behavior(agent):
    """Dragonfly larvae hunt tadpoles; increases speed and turns towards them."""

    left, right = agent.sensors(sensed_entities=["TADPOLES"])

    base_speed = 0.5  # Maintain constant speed
    
    if left > 0 or right > 0:
        base_speed += 0.2
        left_motor = base_speed + (right - left)  # Turn towards right if sensing on left
        right_motor = base_speed + (left - right) # Turn towards left if sensing on right
    else:
        left_motor = base_speed
        right_motor = base_speed  # Move straight if no tadpoles detected

    # Ensure motor values stay within the range [0, 1.0]
    left_motor = max(0.0, min(left_motor, 1.0))
    right_motor = max(0.0, min(right_motor, 1.0))

    agent.diet = ["TADPOLES"] 
    return left_motor, right_motor

In [7]:
import asyncio

async def monitor_hunger(agent):
    """Recursively checks if the dragonfly larvae has eaten in the last 10 seconds.
       If not, removes the agent from the simulation.
    """
    hunger_limit = 10.0  # Max time (seconds) without eating

    while True:
        await asyncio.sleep(50)  # Wait 10 seconds before checking condition

        
        # If hunger limit exceeded, remove the agent
        if agent.has_eaten_since(50) == False:
            agent.exists = False
            return  # Stop the coroutine after removal


In [8]:
tadpoles = [controller.agents[0], controller.agents[1], controller.agents[2], controller.agents[3]]
dragonflies = [controller.agents[4], controller.agents[5]]
for tadpole in tadpoles:
    tadpole.detach_all_behaviors()
    tadpole.attach_behavior(tadpole_behavior)
    tadpole.left_motor = 0.5
    tadpole.right_motor = 0.5
for dragonfly in dragonflies:
    dragonfly.detach_all_behaviors()
    dragonfly.attach_behavior(dragonfly_behavior)
    asyncio.create_task(monitor_hunger(dragonfly))
    dragonfly.left_motor = 0.5
    dragonfly.right_motor = 0.5
controller.start_eating_mechanism()
